In [1]:
import pandas as pd
from collections import Counter
import re
import numpy as np

data=pd.read_csv('data/train_text.csv')


ImportError: No module named 'pandas'

In [110]:
VOCAB_SIZE=20000
MAX_SEQ_LENGTH=250
UNK=VOCAB_SIZE-1
PAD=0

In [111]:
def clean_data(data):
    #replace all other chars except keep_chars with space
    pattern_keep_chars = re.compile('[^a-zA-Z0-9 \']')
    data['comment_text']=data['comment_text'].apply(lambda x : re.sub(pattern_keep_chars,' ',x))
    data['comment_text']=data['comment_text'].apply(lambda x : re.sub(' +',' ',x))
    data['comment_text']=data['comment_text'].apply(lambda x : x.lower())
    data['comment_len']=data['comment_text'].apply(lambda x: len(x.split(' ')))
    #delete senteces with more len 
    prev_len=data.shape[0]
    data=data[data['comment_len']<MAX_SEQ_LENGTH]
    now_len=data.shape[0]
    print('Ignore senteces with more than',MAX_SEQ_LENGTH,'words. prev_len:',prev_len,'now_len:',now_len,'deleted:',prev_len-now_len)
    return data

data=clean_data(data)
print(data.shape)
data.head()


Ignore senteces with more than 250 words. prev_len: 159571 now_len: 152449 deleted: 7122
(152449, 9)


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,comment_len
0,0000997932d777bf,explanation why the edits made under my userna...,0,0,0,0,0,0,47
1,000103f0d9cfb60f,d'aww he matches this background colour i'm se...,0,0,0,0,0,0,19
2,000113f07ec002fd,hey man i'm really not trying to edit war it's...,0,0,0,0,0,0,43
3,0001b41b1c6bb37e,more i can't make any real suggestions on imp...,0,0,0,0,0,0,116
4,0001d958c54c6e35,you sir are my hero any chance you remember wh...,0,0,0,0,0,0,14


In [112]:
#build vocabulary
vocab=data['comment_text'].str.cat(sep=' ').split(' ')
c = Counter(vocab).most_common(VOCAB_SIZE)
vocab= [i[0] for i in c]
vocab[-1]='UNK'
id_to_word=dict(enumerate(vocab))
word_to_id=dict(zip(id_to_word.values(), id_to_word.keys()))

In [113]:
#create array of ints from sentence and padding 
def create_array(s):
    ar=np.zeros(MAX_SEQ_LENGTH)
    for i,w in enumerate(s.split(' ')):
        if(i==250):
            print(s,w,len(s.split(' ')))
        if w in word_to_id:
            ar[i]=word_to_id[w]
        else:
            ar[i]=word_to_id['UNK']
    return ar

data['comment_text']=data['comment_text'].apply(create_array)
col_names=['w'+str(i) for i in range(1,251)]
data[col_names]=pd.DataFrame(data.comment_text.values.tolist(), index= data.index)
X=data[col_names]
X[:10]

,w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,...,w241,w242,w243,w244,w245,w246,w247,w248,w249,w250
0,667.0,72.0,0.0,116.0,132.0,192.0,28.0,615.0,4172.0,10286.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,19999.0,56.0,2370.0,13.0,811.0,3924.0,69.0,4660.0,2835.0,23.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,405.0,415.0,69.0,126.0,14.0,244.0,1.0,68.0,329.0,71.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2.0,57.0,6.0,222.0,97.0,53.0,331.0,1381.0,15.0,1959.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,1445.0,19.0,28.0,3190.0,53.0,980.0,4.0,579.0,39.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2.0,2152.0,33.0,35.0,17.0,99.0,87.0,0.0,2124.0,99.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,6983.0,145.0,4.0,2948.0,311.0,15.0,28.0,139.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,20.0,188.0,1.0,0.0,3905.0,19999.0,22.0,44.0,52.0,275.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,227.0,21.0,0.0,299.0,19999.0,24.0,1390.0,1.0,4.0,458.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,9555.0,15.0,13.0,239.0,3.0,58.0,19.0,1812.0,1.0,151.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [114]:
Y=data[['toxic','severe_toxic','obscene','threat','insult','identity_hate']]
Y[:10]


,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
5,0,0,0,0,0,0
6,1,1,1,0,1,0
7,0,0,0,0,0,0
8,0,0,0,0,0,0
9,0,0,0,0,0,0


In [115]:
X_mat=X.as_matrix()
Y_mat=Y.as_matrix()

In [123]:
test_size=30000

X_train,X_test=X_mat[:-test_size],X_mat[-test_size:]
Y_train,Y_test=X_mat[:-test_size],X_mat[-test_size:]

In [124]:
# train Utils
def get_batch(X,Y,idx,batch_size=400):
    return X[idx*batch_size:(idx+1)*batch_size],Y[idx*batch_size:(idx+1)*batch_size]

In [127]:
from attention import attention
import tensorflow as tf
from tensorflow.contrib.rnn import GRUCell
from tensorflow.python.ops.rnn import dynamic_rnn as rnn
from tensorflow.python.ops.rnn import bidirectional_dynamic_rnn as bi_rnn